## Building an ETL Pipeline for Cryptocurrency data.

- Begin by drawing data in **CSV** format.
- Transform/Manipulate the Data.
- Load Data into an SQL Database.

- Data Source --- https://raw.githubusercontent.com/diljeet1994/Python_Tutorials/master/Projects/Advanced%20ETL/crypto-markets.csv

In [ ]:
import os

# Current working directory
cwd = os.getcwd()
cwd

In [ ]:
# Import all the Pyforest utility Libraries.
from pyforest import *

# Retrieve raw data using Pandas.
crypto_df = pd.read_csv(f"{cwd}/crypto-markets.csv")
crypto_df.head(10)


- The prices of these Currencies are in USD. 
- We need to convert [BTC, ETH, XRP, LTC] currency values into YEN.
- This then calls for The transformation process.
    
- We shall iterate through the price columns [open, close, high, low] and multiply values with 134.56





In [ ]:
currency_code = ['BTC','ETH','XRP','LTC']

crypto_df['open'] = crypto_df[['open','asset']].apply(lambda x: (float(x[0]) * 134.56) if x[1] in currency_code else np.nan, axis=1)
crypto_df['close'] = crypto_df[['close','asset']].apply(lambda x: (float(x[0]) * 134.56) if x[1] in currency_code else np.nan, axis=1)
crypto_df['low'] = crypto_df[['low','asset']].apply(lambda x: (float(x[0]) * 134.56) if x[1] in currency_code else np.nan, axis=1)
crypto_df['high'] = crypto_df[['high','asset']].apply(lambda x: (float(x[0]) * 134.56) if x[1] in currency_code else np.nan, axis=1)

# drop entries with null values
crypto_df.dropna(inplace=True)

# reset the dataframe index
crypto_df.reset_index(drop=True, inplace=True)
crypto_df.head(20)

In [ ]:
crypto_df

In [ ]:
# Lets filter only the data-points that we find relevant. [asset, name, open, high, low, close].
# We drop other columns

crypto_df = crypto_df[['asset','name','date','open','high','low','close']]
crypto_df.head(20)


- Having done the relevant transformations, it is then time to load the data into an SQL Database (sqlite3).

In [ ]:
import sqlite3

# open connection to the database file.
conn = sqlite3.connect('session.db')
print(conn)


In [ ]:
# Drop the table name you would like to assign to your table pif it exists.

try:
    conn.execute('DROP TABLE IF EXISTS `Crypto_data`')
except Exception as e:
    raise(e)
finally:
    # ascertain the table has been dropped
    print("Dropped table")

In [ ]:
# Create a table called `Crypto_data`
try:
    conn.execute('''
    CREATE TABLE Crypto_data(
    ASSET   TEXT,
    NAME    TEXT NOT NULL,
    Date    datetime,
    Open     Float DEFAULT 0,
    High    Float DEFAULT 0,
    Low     Float DEFAULT 0,
    Close   Float DEFAULT 0
    );
    ''')
    print("Table Created")

except Exception as e:
    print(str(e))
    print("Process Failed!")
finally:
    # Close the Database connection.
    conn.close()

In [ ]:
# Convert the pandas dataframe into a list, to allow for the 
# sqlite module to allow insertion of data.

crypto_list = crypto_df.values.tolist()
crypto_list

In [ ]:
# Execute a connection to the database.
conn = sqlite3.connect('session.db')

# create a cursor, this helps with quering the sql database.
cur = conn.cursor()

try:
    cur.executemany("INSERT INTO Crypto_data (ASSET, NAME, Date, Open, High, Low, Close) VALUES (?,?,?,?,?,?,?)", crypto_list)
    conn.commit()
    print('Data Inserted Successfully')
except Exception as e:
    print(str(e))
    raise(e)
    print("Insertion Failed!!")
finally:
    # close the DB connection.
    conn.close()